In [160]:
import pandas as pd
import numpy as np

df = pd.read_csv('penguins.csv')
df=df.dropna()
del df["year"]
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male
...,...,...,...,...,...,...,...
339,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male
340,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female
341,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male
342,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male


In [161]:
main_statistics=df.describe()
print(main_statistics)

       bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g
count      333.000000     333.000000         333.000000   333.000000
mean        43.992793      17.164865         200.966967  4207.057057
std          5.468668       1.969235          14.015765   805.215802
min         32.100000      13.100000         172.000000  2700.000000
25%         39.500000      15.600000         190.000000  3550.000000
50%         44.500000      17.300000         197.000000  4050.000000
75%         48.600000      18.700000         213.000000  4775.000000
max         59.600000      21.500000         231.000000  6300.000000


In [162]:
df['species'] = pd.Categorical(df['species'])
df['island'] = pd.Categorical(df['island'])
df['sex'] = pd.Categorical(df['sex'])
df['species']=df['species'].cat.codes
df['island']=df['island'].cat.codes
df['sex']=df['sex'].cat.codes
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.1,18.7,181.0,3750.0,1
1,0,2,39.5,17.4,186.0,3800.0,0
2,0,2,40.3,18.0,195.0,3250.0,0
4,0,2,36.7,19.3,193.0,3450.0,0
5,0,2,39.3,20.6,190.0,3650.0,1
...,...,...,...,...,...,...,...
339,1,1,55.8,19.8,207.0,4000.0,1
340,1,1,43.5,18.1,202.0,3400.0,0
341,1,1,49.6,18.2,193.0,3775.0,1
342,1,1,50.8,19.0,210.0,4100.0,1


In [163]:
#Normalizenon-categoricalfeatures
df_normal=df
features = df_normal[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]
min_value=features.min()
max_value=features.max()
features=(features-min_value)/(max_value-min_value)
df_normal[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']] = features
df_normal

X=df_normal.drop(['island'], axis=1)
y=df_normal[['island']] #target value

df = df.sample(frac = 1) #shuffle data
train_size = int(0.8 * len(df_normal))
train_X = X[:train_size]
train_y= y[:train_size]
test_X=X[train_size:]
test_y=y[train_size:]

t_X_shpe=train_X.shape
t_y_shpe=train_y.shape
tes_X_shpe=test_X.shape
tes_y_shpe=test_y.shape
print(t_X_shpe,t_y_shpe,tes_X_shpe,tes_y_shpe)

(266, 6) (266, 1) (67, 6) (67, 1)


In [164]:
# select island as the target value Y

In [165]:
#one-hot-encoding
#species
# df_1=df
# add_columns_species = pd.get_dummies(df_1['species'])
# df_1 = df_1.join(add_columns_species)
# df_1.drop(['species'], axis=1, inplace=True)
# #sex
# add_columns_sex = pd.get_dummies(df_1['sex'])
# df_1 =df_1.join(add_columns_sex)
# df_1.drop(['sex'], axis=1, inplace=True)
# df_1[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']] = features
# df_1.head()

In [166]:
class LogitRegression():
    def __init__(self, learning_rate=1e-6, num_iterations=10000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations

    def sigmoid(self,z):
        return 1 / (1 + np.exp(-z))
    
    def cost(self, X, y):
        n = y.shape[0]
        h = self.sigmoid(np.dot(X, self.w.T) )
        J = (1 / n) * (-y.iloc[:,0]*np.log(h)-(1-y.iloc[:,0])*np.log(1-h))
        return J
    
    def gradient_descent(self, X, y):
        n = y.shape[0]
        pred = self.sigmoid(np.dot(X, self.w.T))
        delta = pred - y.iloc[:,0]
        dw = (1 / n) * np.dot(X.T, delta)
        return dw

    def fit(self, X, y):
        # Initialize the weights
        self.w = np.random.uniform(0, 1, X.shape[1])
        # Initialize the loss array
        self.loss = []
        # Run gradient descent for the specified number of iterations
        for i in range(self.num_iterations):
            dw = self.gradient_descent(X, y)
            self.w -= self.learning_rate * dw
            # Calculate the cost function and append it to the loss array
            J = self.cost(X, y)
            self.loss.append(J)

    def predict(self, X):
        # Predict the binary labels for the input data
        y_pred = self.sigmoid(np.dot(X, self.w))
        return np.where(y_pred >= 0.5, 1, 0)
    

In [167]:
model = LogitRegression()

In [168]:
model.fit(train_X, train_y)

In [169]:
train_X


,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,0.254545,0.666667,0.152542,0.291667,1
1,0,0.269091,0.511905,0.237288,0.305556,0
2,0,0.298182,0.583333,0.389831,0.152778,0
4,0,0.167273,0.738095,0.355932,0.208333,0
5,0,0.261818,0.892857,0.305085,0.263889,1
...,...,...,...,...,...,...
272,2,0.534545,0.142857,0.728814,0.597222,0
273,2,0.665455,0.309524,0.847458,0.847222,1
274,2,0.476364,0.202381,0.677966,0.694444,0
275,2,0.647273,0.357143,0.694915,0.750000,1


In [170]:
train_y

,island
0,2
1,2
2,2
4,2
5,2
...,...
272,0
273,0
274,0
275,0


In [171]:
model.predict(test_X)
test_X.iloc[[0]]

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
277,1,0.650909,0.761905,0.40678,0.333333,1


In [172]:
model.w


array([0.73475677, 0.95563987, 0.40888351, 0.60690951, 0.37950135,
       0.50911616])